<a href="https://colab.research.google.com/github/atcuthbertson/CNRA_OpenData_Accessing/blob/main/BasicQuerying_CNRA_OpenData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook demonstrating some basic querying of CNRA Resources using the Datastore API

# Introduction

## Documentation on the APIs used:

https://docs.ckan.org/en/2.9/api/index.html

## Limits to Queries
The CNRA Open Data platform has a return limit of 50,000 records from a query* (subject to change)

## Get an API Key

How to get a CNRA Open DAta API Key (needed for making advanced SQL Queries).  
    
1. Create an account in CNRA Open Data.  Go here: https://data.cnra.ca.gov/user/login  and click 'Create Account' on lefthand side of window.

2. Once you have an account, you are immediately issued an API Key.  Go to your User page (in the toolbar on the top right, you will see an icon and your initials, click this to go to your user page.  You will find your API key on the lefthand column on your user page.


### IMPORTANT:  For examples 3 and 4 in this notebook to work, you will need to get your own API Key and place it in the cell below, on the line 13 key=\<\<your api key here\>\>  ###

For example, if your api key is '12345678-9abc-def1-2345-67890abcdef1', then line 13 should be changed to:
key='12345678-9abc-def1-2345-67890abcdef1'

In [ ]:
#get and store in variables some need information for the queries



# also need resource_ids for resources in Open Data we want to query
#resource IDs in CNRA Open Open Data
resource_measurements = 'bfa9f262-24a1-45bd-8dc8-138bc8107266'
resource_stations = 'af157380-fb42-4abf-b72a-6f9f98868077'
resource_perforations = 'f1deaa6d-2cb5-4052-a73f-08a69f26b750'

## Example 1: Basic search (datastore_search) -- get first 5 records of a resource

In [ ]:
#set up url for basic search for first 5 records of resource, using resource_id and limit parameters
searchUrl = urlBasicSearch + '?resource_id=' + resourceLabResults + '&limit=5' + '&sort="sample_date" '
searchUrl

NameError: name 'urlBasicSearch' is not defined

In [ ]:
# call the api using requests.get
results = requests.get(searchUrl)

#Parse the returned result json to get the results into a dataframe -- lots of ways to do this.  Here is one that works for me.
df = pd.read_json(results.content)
resultsList = df['result']['records']
resultsTable = pd.json_normalize(resultsList) # normalize the resulting
resultsTable

In [ ]:
#Need apiKey for sql queries using datastore_search_sql calls
# apikey  -- stored in secrets. Replace this with your api key.

from google.colab import userdata
apikey= userdata.get('cnraOpenDataApiKey')

# apikey = 'your-api-key-here'

In [ ]:
# prompt: function to request data using basic query from ckan datastore

def query_ckan_datastore(resource_id, query):
    """Queries a CKAN datastore resource using a basic query.

    Args:
        resource_id: The ID of the CKAN datastore resource.
        query: The SQL-like query string.

    Returns:
        A list of dictionaries representing the query results,
        or None if there's an error.
    """

    base_url = "https://data.cnra.ca.gov/api/3/action/datastore_search_sql"  # Replace with your CKAN instance's base URL
    params = {
        "sql": query
    }
    headers = {'Authorization': 'your_api_key'} # If needed, replace with your API key
    try:
        response = requests.get(base_url, params=params, headers = headers)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()

        if data["success"]:
          return data["result"]["records"]
        else:
          print(f"Error querying datastore: {data['error']}")
          return None
    except requests.exceptions.RequestException as e:
        print(f"Error querying CKAN datastore: {e}")
        return None

In [ ]:



# Example usage (replace with your resource ID and query)
resource_id = resource_stations  # groundwater monitoring stations
query = f"SELECT site_code, gse FROM \"{resource_id}\" LIMIT 10" # Example query, replace with your query
results = query_ckan_datastore(resource_id, query)


if results:
    # Process the results
    #for record in results:
    #  record

    # Or convert to a Pandas DataFrame:
    #import pandas as pd
    df = pd.DataFrame(results)
    df


In [ ]:
df

,site_code,gse
0,320000N1140000W001,545.92000
1,325450N1171061W001,17.14000
2,325450N1171061W002,17.14000
3,325450N1171061W003,17.14000
4,325450N1171061W004,17.14000
5,325450N1171061W005,17.14000
6,325493N1170884W001,28.16000
7,325536N1170608W001,45.26000
8,325538N1170626W001,37.18000
9,325538N1170626W002,37.18000


# Getting started - preliminary set up

## Import packages and set up some constants to streamline testing




In [ ]:
# import libraries for examples

import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

# Example 1:  Searching for datasets in the filestore

# Example 2:  Very Basic query of a datastore resource

# Example 3:  SQL queriying of datastore resource - simple

# Example 4:  SQL query - yes, you can do joins!

# New Section